<a href="https://colab.research.google.com/github/IAmSuyogJadhav/Brainy/blob/master/Build_Brainy_Dependencies_for_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brainy Dependencies


# Mount Drive

In [0]:
from google.colab import drive
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


# Build ANTS for nipype


## Download and extract ANTS

In [0]:
%cd /content/
!apt-get install cmake-curses-gui build-essential zlib1g-dev
!mkdir code
%cd code
!git clone https://github.com/stnava/ANTs.git
%cd ..
%mkdir -p bin/ants
%cd bin/ants

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  cmake-curses-gui
0 upgraded, 1 newly installed, 0 to remove and 10 not upgraded.
Need to get 1,520 kB of archives.
After this operation, 5,049 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 cmake-curses-gui amd64 3.10.2-1ubuntu2 [1,520 kB]
Fetched 1,520 kB in 1s (1,957 kB/s)
Selecting previously unselected package cmake-curses-gui.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../cmake-curses-gui_3.10.2-1ubuntu2_amd64.deb ...
Unpacking cm

## Configure ccmake

### Get public SSH address for Colaboratory VM
Needed to run `ccmake` only. Can't be done from Colaboratory directly, due to its limited command-line interface.

In [0]:
#Generate root password
import random, string
# password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(7))
password = 'coldsnake'  # Can change this

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('SSH to: root@', json.load(sys.stdin)['tunnels'][0]['public_url'][6:], sep='')"
#Print root password
print("Root password: {}".format(password))

SSH to: root@0.tcp.ngrok.io:13625
Root password: coldsnake


In [0]:
%cd /content/
with open('run.sh', 'w') as f:
  f.write(
  """
  cd /content/bin/ants
  ccmake /content/code/ANTs
  """.strip()
  )
  

### Now Do following from your computer
Use the above credentials to connect to Colaboratory VM using ssh. 

1. Run the following
```bash
bash /content/run.sh
```
2. Press the [c] button to configure the compilation options.
3. Turn the tests OFF to save time.
4. Confirm changes by pressing [Return]. Now press [c] and then [g] to save and exit.
5. Terminate the SSH session and continue executing following cells.

## Make

In [0]:
%cd /content/bin/ants

/content/bin/ants


In [0]:
!make -j6  # Sit back and relax, takes a LOT of time.

[ 22%] Linking CXX static library /content/bin/ants/lib/libl_ImageMath.a
[ 22%] Built target l_ImageMath
Scanning dependencies of target l_ConvertImagePixelType
[ 22%] Building CXX object Examples/CMakeFiles/l_ConvertImagePixelType.dir/ConvertImagePixelType.cxx.o
[ 22%] Linking CXX static library /content/bin/ants/lib/libl_WarpTensorImageMultiTransform.a
[ 22%] Built target l_WarpTensorImageMultiTransform
Scanning dependencies of target l_antsJointTensorFusion
[ 23%] Building CXX object Examples/CMakeFiles/l_antsJointTensorFusion.dir/antsJointTensorFusion.cxx.o
[ 23%] Linking CXX static library /content/bin/ants/lib/libl_ConvertImagePixelType.a
[ 23%] Built target l_ConvertImagePixelType
Scanning dependencies of target l_ANTSUseDeformationFieldToGetAffineTransform
[ 23%] Building CXX object Examples/CMakeFiles/l_ANTSUseDeformationFieldToGetAffineTransform.dir/ANTSUseDeformationFieldToGetAffineTransform.cxx.o
[ 24%] Linking CXX static library /content/bin/ants/lib/libl_simpleSynRegistra

## Compress the built ANTS binaries and save to Drive

In [0]:
!7z a ANTS_built_for_colab.7z /content/bin/ants/bin


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Open archive: ANTS_built_for_colab.7z
--
Path = ANTS_built_for_colab.7z
Type = 7z
Physical Size = 614043653
Headers Size = 290961
Method = LZMA2:24 BCJ
Solid = +
Blocks = 2

Scanning the drive:
  0M Scan  /content/bin/ants/                             1 folder, 101 files, 1961813896 bytes (1871 MiB)

Updating archive: ANTS_built_for_colab.7z

Items to compress: 102

  0%      0% = ants/ngrok-stable-linux-amd64.zip                                          1% = ants/ngrok-stable-linux-amd64.zip                                          2% = ants/ngrok-stable-linux-amd64.zi

In [0]:
import shutil
shutil.copy('ANTS_built_for_colab.7z', '/gdrive/My Drive/')

'/gdrive/My Drive/ANTS_built_for_colab.7z'

# Test the Build

Make the ANTS binaries available system-wide

In [0]:
!cp /content/bin/ants/bin/* /usr/local/bin/ -r

**Get an example image**
1. Open [this](https://drive.google.com/file/d/1-2S-OpEeUkisTOq0AIG3bE4EvFJutcme/view?usp=sharing) link.
2. Add the file to your drive.
3. Continue.

In [0]:
shutil.copy('/gdrive/My Drive/test.mha', '/content/')

'/content/test.mha'

In [0]:
from nipype.interfaces.ants import N4BiasFieldCorrection
correct = N4BiasFieldCorrection()
correct.inputs.input_image = '/content/test.mha'
correct.inputs.output_image = '/content/test_corrected.mha'
done = correct.run()

done.outputs.output_image

'/content/t.mha'